In [8]:
// prio_psprio example - aligned with MATLAB implementation
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.ssa.*

In [9]:
// Create network with PS-PRIO scheduling
val model = Network("prio_psprio")

// Block 1: nodes
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.PSPRIO) // PS with priorities
node2.setNumberOfServers(1)

In [10]:
// Block 2: classes with different priorities
val jobclass1 = ClosedClass(model, "Class1", 2, node1, 0)
val jobclass2 = ClosedClass(model, "Class2", 2, node1, 1)

In [11]:
// Set service times - matching MATLAB implementation
node1.setService(jobclass1, Erlang(3.0, 2))
node1.setService(jobclass2, HyperExp(0.5, 3.0, 10.0))

node2.setService(jobclass1, HyperExp(0.1, 1.0, 10.0))
node2.setService(jobclass2, Exp(1.0))

In [12]:
// Block 3: routing - using serialRouting pattern
var P = model.initRoutingMatrix()
P[jobclass1] = Network.serialRouting(node1, node2)
P[jobclass2] = Network.serialRouting(node1, node2)
model.link(P)

In [ ]:
// Solve with different solvers
CTMC(model).avgTable.print()
JMT(model,"seed", 23000,"samples", 5000).avgTable.print()
SSA(model,"seed", 23000,"samples", 5000).avgTable.print()